In [1]:
import pandas as pd

In [2]:
filename = '/content/drive/Othercomputers/My Laptop/data-science-projects/sales_predictions.csv'

In [3]:
#Import a csv file
df = pd.read_csv(filename)
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [5]:
for i in df.columns:
    x,y = df.loc[df[i].isnull()].shape
    if x > 0:
        print("Column {} has {} blank rows".format(i,x))


Column Item_Weight has 1463 blank rows
Column Outlet_Size has 2410 blank rows


In [6]:
df.info

<bound method DataFrame.info of      Item_Identifier  Item_Weight  ...        Outlet_Type  Item_Outlet_Sales
0              FDA15        9.300  ...  Supermarket Type1          3735.1380
1              DRC01        5.920  ...  Supermarket Type2           443.4228
2              FDN15       17.500  ...  Supermarket Type1          2097.2700
3              FDX07       19.200  ...      Grocery Store           732.3800
4              NCD19        8.930  ...  Supermarket Type1           994.7052
...              ...          ...  ...                ...                ...
8518           FDF22        6.865  ...  Supermarket Type1          2778.3834
8519           FDS36        8.380  ...  Supermarket Type1           549.2850
8520           NCJ29       10.600  ...  Supermarket Type1          1193.1136
8521           FDN46        7.210  ...  Supermarket Type2          1845.5976
8522           DRG01       14.800  ...  Supermarket Type1           765.6700

[8523 rows x 12 columns]>

In [7]:
df.shape

(8523, 12)

###Thought about filling in the missing item weights, but it involve setting up a Loop and item weights' does not appear to contribute to predicting the target. This field might actually be dropped later.

In [8]:
#Content conscious customers
#might purchase based on this description, plus field can be converted into 0 
#for regular and 1 for Low Fat. Arriving at two values for this column is a 
#good thing
df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [9]:
#Performing the replace on the improper category descriptions
df.replace({'Item_Fat_Content': {'low fat':'Low Fat','LF':'Low Fat','reg':'Regular'}},inplace=True)

In [10]:
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [11]:
#Checking the Item Type for any necessary category item cleanup
df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

###Figuring out the missing Outlet size was challenging, was not ready to eliminate records simply because the value was missing, given that the population is very small

In [12]:
#The next three cells are ran to see when Grouped by Outlet Location Type, Outlet_Type
#Outlet Establishment Year, which have no Outlet Size, the mean, min and max are
#presented to information the judgement taken on how to fill in the missing
#Outlet Size 
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].mean()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small               340.329723
                     Supermarket Type1 1997                      Small              2277.844267
                                       1999                      Medium             2348.354635
Tier 2               Supermarket Type1 2002                      NaN                2192.384798
                                       2004                      Small              2438.841866
                                       2007                      NaN                2340.675263
Tier 3               Grocery Store     1998                      NaN                 339.351662
                     Supermarket Type1 1987                      High               2298.995256
                     Supermarket Type2 2009                      Medium             1995.498739
                     Supermarket Type3 1985                      Medium             3694.038558

In [13]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].min()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small                  33.9558
                     Supermarket Type1 1997                      Small                 101.8674
                                       1999                      Medium                111.8544
Tier 2               Supermarket Type1 2002                      NaN                    99.8700
                                       2004                      Small                 113.8518
                                       2007                      NaN                   143.8128
Tier 3               Grocery Store     1998                      NaN                    33.2900
                     Supermarket Type1 1987                      High                   73.2380
                     Supermarket Type2 2009                      Medium                 69.2432
                     Supermarket Type3 1985                      Medium                241.6854

In [14]:
df.groupby(['Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year','Outlet_Size'],dropna=False)[['Item_Outlet_Sales']].max()

Item_Outlet_Sales
Outlet_Location_Type Outlet_Type       Outlet_Establishment_Year Outlet_Size                   
Tier 1               Grocery Store     1985                      Small                1482.0708
                     Supermarket Type1 1997                      Small                9779.9362
                                       1999                      Medium               7646.0472
Tier 2               Supermarket Type1 2002                      NaN                  8994.9580
                                       2004                      Small                8479.6288
                                       2007                      NaN                  9664.7528
Tier 3               Grocery Store     1998                      NaN                  1775.6886
                     Supermarket Type1 1987                      High                10256.6490
                     Supermarket Type2 2009                      Medium               6768.5228
                     Supermarket Type3 1985                      Medium              13086.9648

###Based on the presented data, I have made the judgement to update the missing data for Tier 2, Supermarket Type 1, 2002 to Outlet size Small
###Tier 2, Supermarket Type 1, 2007 to Outlet size Medium
###Tier 3, Grocery Store, 1998 to Outlet size Small

In [15]:
tier2_outlet_loc_type_filter = df['Outlet_Location_Type'] == 'Tier 2'
tier3_outlet_loc_type_filter = df['Outlet_Location_Type'] == 'Tier 3'
supermart_type1_filter = df['Outlet_Type'] == 'Supermarket Type1'
grocery_type_filter = df['Outlet_Type'] == 'Grocery Store'
outlet_established_year2002_filter = df['Outlet_Establishment_Year'] == 2002
outlet_established_year1998_filter = df['Outlet_Establishment_Year'] == 1998
outlet_established_year2007_filter = df['Outlet_Establishment_Year'] == 2007

In [16]:
#Code updates cells in column Outlet Size with the determined value
df.loc[outlet_established_year1998_filter | outlet_established_year2002_filter, 'Outlet_Size'] = "Small"
df.loc[outlet_established_year2007_filter, 'Outlet_Size'] = "Medium"


In [17]:
#Repeating this validation to see which columns still have blank cells
for i in df.columns:
    x,y = df.loc[df[i].isnull()].shape
    if x > 0:
        print("Column {} has {} blank rows".format(i,x))


Column Item_Weight has 1463 blank rows


In [23]:
#Determined that item visibility is useful in prediciting target
#Defined a process for updating rows which currently show as zero for
#Item Visibility but have sales

#df_mean and df_mean_item_id are sub dataframes used to determine item visibility
#df_mean is grouped by Item_Identifier and Outlet Size with a mean on
#Item Visibility; df_mean_item_id is similar to df_mean, but does not include
#Outlet_Size
df_mean = df.loc[df['Item_Visibility'] != 0,:]
df_mean = df_mean.groupby(['Item_Identifier','Outlet_Size'],dropna=True,as_index=False)[['Item_Visibility']].mean()

df_mean_item_id = df_mean.groupby(['Item_Identifier'],dropna=True,as_index=False)[['Item_Visibility']].mean()

In [24]:
df_mean

,Item_Identifier,Outlet_Size,Item_Visibility
0,DRA12,High,0.040912
1,DRA12,Medium,0.041145
2,DRA12,Small,0.068535
3,DRA24,High,0.039895
4,DRA24,Medium,0.039960
...,...,...,...
3870,NCZ53,High,0.024457
3871,NCZ53,Medium,0.024484
3872,NCZ53,Small,0.040970
3873,NCZ54,Medium,0.083381


In [25]:
df_mean_item_id

,Item_Identifier,Item_Visibility
0,DRA12,0.050197
1,DRA24,0.046247
2,DRA59,0.148154
3,DRB01,0.082138
4,DRB13,0.007842
...,...,...
1554,NCZ30,0.028129
1555,NCZ41,0.064545
1556,NCZ42,0.011298
1557,NCZ53,0.029970


In [20]:
#Code to update item visibility that is zero based on
#Outlet Size and Item Identifier
for i in range(0,df_mean.shape[0]):
    outlet_size_filter = df['Outlet_Size'] == df_mean['Outlet_Size'][i]
    item_id_filter = df['Item_Identifier'] == df_mean['Item_Identifier'][i]
    item_vis = df_mean['Item_Visibility'][i]
    
    df.loc[outlet_size_filter & item_id_filter, 'Item_Visibility'] = item_vis


In [34]:
#Code to update item visibility that is zero based on item identifier
for i in range(0,df.shape[0]):
    if df.loc[i,'Item_Visibility'] == 0:
        for j in range(0,df_mean_item_id.shape[0]):
            if df_mean_item_id.loc[j,'Item_Identifier'] == df.loc[i,'Item_Identifier']:
                item_vis = df_mean_item_id.loc[j,'Item_Visibility']
                df.loc[i,'Item_Visibility'] = item_vis
                break
            


In [35]:
#Checking that all items have an item visibility value determined
df.loc[df['Item_Visibility'] == 0]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


In [37]:
df['Outlet_Size'].value_counts()

Small     3872
Medium    3719
High       932
Name: Outlet_Size, dtype: int64

#First Checkpoint
##Checkpoint is provided just before performing removal of any duplicate rows

In [38]:
df1_copy = df

In [39]:
df.drop_duplicates()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016048,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019231,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016738,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.022940,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.015437,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046916,Baking Goods,108.1570,OUT045,2002,Small,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035225,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145038,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [40]:
df

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016048,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019231,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016738,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.022940,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.015437,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046916,Baking Goods,108.1570,OUT045,2002,Small,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035225,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145038,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976
